In [1]:
import pandas as pd
from moviepy.editor import *
from PIL import Image
from PIL import ImageDraw
import numpy as np

In [2]:
# this can be empty if the video file and its videopipe output are at the same
# location as the code
path = ''
v_name = 'HIGH_LIGHTS_I_SNOWMAGAZINE_I_SANDER_26'
task = '_image_aesthetics_datamodel'
w, h = 1920, 1080

In [3]:
## read shots detection json

aesthetics = pd.read_json(path + v_name + '/' + v_name + task + '.json', lines = True)
aesthetics_detected = [f for f in aesthetics.data[0]]

In [4]:
## read video file

clip = VideoFileClip(v_name + '.mp4')

fps = clip.fps
frame_duration = 1 / fps

Set n to how many frames the top frames should consist of.

In [5]:
# Set the amount of frames
n = 10

Get the top n aesthetics frames and the top n technical frames, ordered from lowest to highest. Also get the top frames for both aesthetics and technical.

In [6]:
## Get 2n frames from aesthetics_detected, n with the highest aesthetics score and n with the highest technical score.

highest_aesthetics = sorted(aesthetics_detected, key = lambda x: x['aesthetics_score'], reverse = True)
highest_technical = sorted(aesthetics_detected, key = lambda x: x['technical_score'], reverse = True)

# Get highest of both.
highest = sorted(aesthetics_detected, key = lambda x: x['aesthetics_score'] + x['technical_score'], reverse = True)

# Top frames from lowest to highest
top_aesthetics = [f['dimension_idx'] for f in reversed(highest_aesthetics[:n])]
top_technical = [f['dimension_idx'] for f in reversed(highest_technical[:n])]
top_both = [f['dimension_idx'] for f in reversed(highest[:n])]

print('Top ' + str(n) + ' aesthetics frames: ', top_aesthetics)
print('Top ' + str(n) + ' technical frames: ', top_technical)
print('Top ' + str(n) + ' both frames: ', top_both)

Top 10 aesthetics frames:  [4186, 4004, 2184, 2002, 5096, 13832, 2366, 3276, 3094, 4368]
Top 10 technical frames:  [4368, 6916, 6552, 4550, 8190, 9464, 7462, 7826, 6188, 10374]
Top 10 both frames:  [4914, 9464, 6916, 16016, 6188, 4186, 2002, 8190, 4550, 4368]


In [7]:
def get_frame(clip, frame_number):
    return Image.fromarray(clip.get_frame(frame_number * frame_duration))

Create image clips which are prefixed with a textclips showing the ranking of the frame.

In [8]:
# Set duration of frame.
duration_f = 3

# Set duration of text
duration_t = 1


aesthetics_clips = []
technical_clips = []
both_clips = []

# Create the list of textclips and imageclips.
def subclips(top, duration_fr, duration_txt, count):
    clips = []
    for f in top:
        txtclip = TextClip('Frame ' + str(count), fontsize = 50, color = 'white', size=(w,h)).set_duration(duration_txt)
        frame = get_frame(clip, f)
        imgclip = ImageClip(np.asarray(frame), duration = duration_fr)
        clips.append(txtclip)
        clips.append(imgclip)
        count -= 1
    return clips

aesthetics_clips = subclips(top_aesthetics, duration_f, duration_t, n)
technical_clips = subclips(top_technical, duration_f, duration_t, n)
both_clips = subclips(top_both, duration_f, duration_t, n)

Create the gifs, fps can be set to 1 to improve performance since we only show still images

In [9]:
final_aesthetics = concatenate_videoclips(aesthetics_clips)
final_technical = concatenate_videoclips(technical_clips)
final_both = concatenate_videoclips(both_clips)

final_aesthetics.write_gif("aesthetics.gif", fps=1)
final_technical.write_gif("technical.gif", fps=1)
final_both.write_gif("both.gif", fps=1)

MoviePy - Building file aesthetics.gif with imageio.


MoviePy - Building file technical.gif with imageio.


MoviePy - Building file both.gif with imageio.
